In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import os
from glob import glob
from matplotlib.dates import DateFormatter

def format_title(filename):
    # Remove .csv extension
    filename = filename.replace('.csv', '')
    
    # Split into tokens
    tokens = filename.split('_')
    
    # Remove date
    tokens = tokens[1:]
    
    # Handle first token special cases
    if tokens[0].lower() == 'yoyo':
        tokens[0] = 'YoYo'
    elif tokens[0].lower() == 'ddos':
        tokens[0] = 'DDoS'
    else:
        tokens[0] = tokens[0].capitalize()
    
    # Capitalize remaining tokens if they start with a letter
    for i in range(1, len(tokens)):
        if tokens[i][0].isalpha():
            tokens[i] = tokens[i].capitalize()
    
    # Format the time unit token
    last_token = tokens[-1]
    if last_token[-1] in ['m', 'h']:
        number = last_token[:-1]
        unit = last_token[-1]
        tokens[-1] = f'({number}{unit})'
    
    tokens.append(', Randomized Autoscaling')

    return ' '.join(tokens)

# Create a directory for plots if it doesn't exist
plots_dir = 'plots'
if not os.path.exists(plots_dir):
    os.makedirs(plots_dir)

# Get all CSV files in the current directory
csv_files = glob('*.csv')

# Create plots for each CSV file
for csv_file in csv_files:

# csv_file = '250116_no_attacker_service_units_used_1m.csv'

    # Read the CSV file with header
    data = pd.read_csv(csv_file)
        
    # Get column names
    x_col = data.columns[0]
    y_col = data.columns[1]

    # Convert the x-axis to datetime
    data[x_col] = pd.to_datetime(data[x_col])

    # Calculate elapsed time in minutes from the first timestamp
    data['Elapsed Minutes'] = (data[x_col] - data[x_col].iloc[0]).dt.total_seconds() / 60

    # Create the plot
    plt.figure(figsize=(12, 6))
    plt.plot(data['Elapsed Minutes'], data[y_col], marker='.', markersize=3, linestyle='-')
    plt.title(format_title(csv_file))
    plt.xlabel('Elapsed Time (minutes)')
    plt.ylabel(y_col)

    # Set x-axis limits from 0 to 120 minutes
    plt.xlim(0, 120)

    # Set ticks every 10 minutes
    plt.xticks(range(0, 121, 10))

    # Save the plot with tight layout
    plt.tight_layout()
    plt.savefig(os.path.join(plots_dir, f'{csv_file[:-4]}.png'))
    plt.close()